In [5]:
# -*- coding: utf-8 -*-
"""
Ökobilanz-Rechner für Deponiegasverwertung durch einen Dual-Fuel-Motor.

Diese Anwendung berechnet die ökologischen Auswirkungen basierend auf Nutzer-Inputs
für Gasmenge, Methangehalt und Betrachtungszeitraum.
"""

import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
import io
import requests

# ----------------------------------------------------------------
# GitHub Raw-URLs for your CSV files
# ----------------------------------------------------------------
URL_DUAL_FUEL_BACKGROUND = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/1.Dual_Fuel_Background_Data.csv"
URL_DUAL_FUEL_IMPACTS    = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/2.Dual_Fuel_Impacts.csv"
URL_EMISSION_IMPACTS     = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/3.Emission_Impacts.csv"
URL_ENGINE_DATA          = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/4.Engine_Data.csv"
URL_ASSUMPTIONS          = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/5.Assumptions.csv"
# CORRECTED URL: Using the "raw" version of the file, not the HTML page
URL_EMISSIONS            = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/6.Emission.csv"


# ----------------------------------------------------------------
# Class for the calculation
# ----------------------------------------------------------------
class LCA_Calculator:
    def __init__(self):
        """
        Initializes the calculator and loads the CSV files directly from GitHub.
        """
        self.impact_categories = []

        # FIXED: The data loading block (try...except) must be INSIDE the __init__ method
        # so it can access 'self'.
        try:
            # --- Load data ---
            self.df_dual_fuel_background = pd.read_csv(
                URL_DUAL_FUEL_BACKGROUND,
                sep=";", header=[0, 1], index_col=0
            )

            self.df_dual_fuel_impacts = pd.read_csv(
                URL_DUAL_FUEL_IMPACTS,
                sep=";", index_col=0
            )

            self.df_emission_impacts = pd.read_csv(
                URL_EMISSION_IMPACTS,
                sep=";", index_col=0, skiprows=[1]  # Skip "Unit" row
            )

            self.df_engine_data = pd.read_csv(
                URL_ENGINE_DATA,
                sep=";", index_col=0
            )
            
            # Clean up empty columns from the engine data CSV
            self.df_engine_data = self.df_engine_data.dropna(axis=1, how='all')


            self.df_assumptions = pd.read_csv(
                URL_ASSUMPTIONS,
                sep=";", index_col=0, header=None
            ).squeeze("columns").to_dict()

            self.df_emissions = pd.read_csv(
                URL_EMISSIONS,
                sep=";"
            )
            # Set index after loading
            self.df_emissions.set_index(self.df_emissions.columns[0], inplace=True)
            # Clean up empty columns from the emissions CSV
            self.df_emissions = self.df_emissions.dropna(axis=1, how='all')


            # --- Clean data ---
            self._clean_data()
            self.impact_categories = self.df_dual_fuel_impacts.columns.tolist()

        except Exception as e:
            # This exception will be caught by the App class and shown in a message box.
            raise RuntimeError(f"Fehler beim Laden der Daten aus GitHub: {e}")


    def _clean_data(self):
        """Cleans column names in the DataFrames."""
        for df in [self.df_dual_fuel_impacts, self.df_emission_impacts]:
            # Remove leading/trailing whitespace from column names
            df.columns = df.columns.str.strip()

    def find_nearest_load(self, target_load, load_array):
        """Finds the nearest value in a given array."""
        idx = (np.abs(np.array(load_array) - target_load)).argmin()
        return load_array[idx]

    def run_calculation(self, initial_gas_volume, initial_methane_content, years, manual_gas=None, manual_methane=None):
        """
        This is a placeholder for your actual calculation logic.
        """
        # TODO: Replace this dummy implementation with your real calculation logic.
        print("--- Running Calculation ---")
        print(f"Initial Gas Volume: {initial_gas_volume}, Methane Content: {initial_methane_content}, Years: {years}")
        print("\n--- Loaded DataFrames (Head) ---")
        print("\nDual Fuel Background:")
        print(self.df_dual_fuel_background.head())
        print("\nEngine Data:")
        print(self.df_engine_data.head())
        print("\nEmissions:")
        print(self.df_emissions.head())
        print("\nAssumptions:")
        print(self.df_assumptions)
        print("\n--------------------------")
        
        # Dummy return for testing purposes
        return {
            "selected_engine": "ExampleEngine",
            "valid_years": years,
            "total_impact": {"Global warming": 1234.56, "Fossil resource scarcity": 789.01},
            "yearly_impacts": {y: {"Global warming": 200 * y, "Fossil resource scarcity": 150 * y} for y in range(1, years + 1)}
        }


# ----------------------------------------------------------------
# GUI class with Tkinter
# ----------------------------------------------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Ökobilanz-Rechner für Deponiegasverwertung")
        self.geometry("1200x800")
        self.config(bg="#f0f0f0")

        # --- Initialize Calculator ---
        try:
            self.calculator = LCA_Calculator()
        except Exception as e:
            messagebox.showerror("Fehler beim Initialisieren", f"Die Anwendung konnte nicht gestartet werden.\n\n{e}")
            self.destroy()
            return

        # --- GUI Elements ---
        main_frame = ttk.Frame(self, padding="20")
        main_frame.pack(expand=True, fill="both")

        title_label = ttk.Label(main_frame, text="Ökobilanz-Rechner", font=("Arial", 24, "bold"))
        title_label.pack(pady=(0, 20))

        # --- Input Frame ---
        input_frame = ttk.Frame(main_frame, padding="10", relief="groove")
        input_frame.pack(pady=10, fill="x")

        ttk.Label(input_frame, text="Gasmenge [m³/h]:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.gas_volume_entry = ttk.Entry(input_frame)
        self.gas_volume_entry.grid(row=0, column=1, padx=5, pady=5)
        self.gas_volume_entry.insert(0, "1000")

        ttk.Label(input_frame, text="Methangehalt [%]:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.methane_content_entry = ttk.Entry(input_frame)
        self.methane_content_entry.grid(row=1, column=1, padx=5, pady=5)
        self.methane_content_entry.insert(0, "50")

        ttk.Label(input_frame, text="Betrachtungszeitraum [Jahre]:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.years_entry = ttk.Entry(input_frame)
        self.years_entry.grid(row=2, column=1, padx=5, pady=5)
        self.years_entry.insert(0, "5")

        # --- Action Button ---
        calc_button = ttk.Button(main_frame, text="Berechnung starten", command=self._run_calculation_from_gui)
        calc_button.pack(pady=20)

        # --- Results Display ---
        self.results_text = tk.Text(main_frame, height=20, width=100, font=("Courier New", 10), state="disabled")
        self.results_text.pack(pady=10, expand=True, fill="both")

    def _run_calculation_from_gui(self):
        """Gets values from the GUI, runs the calculation, and displays the results."""
        try:
            gas_volume = float(self.gas_volume_entry.get())
            methane_content = float(self.methane_content_entry.get())
            years = int(self.years_entry.get())
            
            if gas_volume <= 0 or methane_content <= 0 or years <= 0:
                messagebox.showwarning("Ungültige Eingabe", "Bitte geben Sie nur positive Zahlen ein.")
                return

            results = self.calculator.run_calculation(gas_volume, methane_content, years)
            
            # Format and display results
            output = f"BERECHNUNGSERGEBNISSE\n"
            output += f"{'='*30}\n"
            output += f"Ausgewählter Motor: {results.get('selected_engine', 'N/A')}\n"
            output += f"Gültige Jahre: {results.get('valid_years', 'N/A')}\n"
            output += f"\n--- Gesamtauswirkungen ---\n"
            
            total_impact = results.get('total_impact', {})
            if total_impact:
                for category, value in total_impact.items():
                    output += f"{category:<30}: {value:,.2f}\n"
            else:
                output += "Keine Gesamtdaten verfügbar.\n"

            output += f"\n--- Jährliche Auswirkungen ---\n"
            yearly_impacts = results.get('yearly_impacts', {})
            if yearly_impacts:
                 for year, impacts in yearly_impacts.items():
                     output += f"\nJahr {year}:\n"
                     for category, value in impacts.items():
                         output += f"  - {category:<28}: {value:,.2f}\n"

            self.results_text.config(state="normal")
            self.results_text.delete("1.0", tk.END)
            self.results_text.insert(tk.END, output)
            self.results_text.config(state="disabled")

        except ValueError:
            messagebox.showerror("Eingabefehler", "Bitte stellen Sie sicher, dass alle Felder korrekt ausgefüllt sind (Zahlen).")
        except Exception as e:
            messagebox.showerror("Berechnungsfehler", f"Ein unerwarteter Fehler ist aufgetreten:\n\n{e}")

# ----------------------------------------------------------------
# Main entry point
# ----------------------------------------------------------------
if __name__ == "__main__":
    app = App()
    app.mainloop()

CHECK_PFPROJ: non-valid_pfProj (pfProj = 0.992249)
CHECK_PFPROJ: non-valid_pfProj (pfProj = 0.993286)
